# YOLO v4 demo (MS COCO trained)

Choose GPU for faster inference

**Runtime -> Change Runtime Type -> GPU**

In [1]:
#@title Print GPU info
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Sat Dec 19 03:28:07 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.45.01    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   63C    P8    10W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
#@title Download darknet
# Clone darknet
!git clone https://github.com/AlexeyAB/darknet.git
# For downloading & trimming vids
!git clone https://github.com/ankandrew/YT-Downloader-Trimmer.git ytdownloader

!pip install -r /content/ytdownloader/requirements.txt

from ytdownloader.downloader import Downloader
from google.colab import files

%cd darknet

!wget https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v3_optimal/yolov4.weights
!wget https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v4_pre/yolov4-tiny.weights

# Enable GPU & OPENCV
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!sed -i 's/CUDNN_HALF=0/CUDNN_HALF=1/' Makefile

# Make
!make

!chmod 755 darknet

In [ ]:
#@title Download YT video for making predictions
video_url = "https://www.youtube.com/watch?v=Zbm0Z6nw5sE" #@param {type:"string"}
video_quality = 720 #@param ["1080", "720", "480", "360", "240"] {type:"raw"}
#@markdown Only video w/o sound
only_video = True #@param {type:"boolean"}
#@markdown Trim video
do_trim = True #@param {type:"boolean"}
#@markdown Time in hh:mm:ss format
start = "00:13:57" #@param {type:"string"}
end = "00:15:00" #@param {type:"string"}

yt_d = Downloader(
    video_url, 'downloaded_vid.mp4', quality=video_quality, only_vid=only_video)
yt_d.download()

if do_trim:
  import os
  yt_d.trim(start, end, delete_original=False)
  os.rename('downloaded_vid_trimmed.mp4', 'downloaded_vid.mp4')

In [ ]:
#@title Or upload video from PC
file_name = "Name of the file.mp4" #@param {type:"string"}

assert file_name, 'Fill in the name of the file to upload'

files.upload()

assert os.path.isfile(file_name), 'Uploaded file has different name'
os.rename(file_name, 'downloaded_vid.mp4')

### Yolo v4

In [ ]:
#@title Run YOLO v4
!./darknet detector demo cfg/coco.data cfg/yolov4.cfg yolov4.weights downloaded_vid.mp4 -dont_show -out_filename res_yolov4.mp4

### Yolo v4 - tiny

In [ ]:
#@title Run YOLO v4 tiny
!./darknet detector demo cfg/coco.data cfg/yolov4-tiny.cfg yolov4-tiny.weights downloaded_vid.mp4 -dont_show -out_filename res_yolov4_tiny.mp4

### Download processed vid

In [ ]:
#@title Download video
option = "yolo v4 tiny" #@param ["yolo v4", "yolo v4 tiny"]
# Download file
if option == 'yolo v4':
  files.download('res_yolov4.mp4')
else:
  files.download('res_yolov4_tiny.mp4')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>